In [2]:
import cv2 as cv
import mediapipe as mp
from models.hand_gesture import HandGesture
from models.hand_gesture import landmark_bbox, normalized_landmarks

In [3]:
PATH = 'data/videos/hand_shake.mp4'
PATH_GESTURE_MODEL = 'models/hand_gesture_model.pth'

In [4]:
video = cv.VideoCapture(PATH)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

Trail = []
my_gesture = HandGesture(PATH_GESTURE_MODEL)
while video.isOpened():
    ret, frame = video.read()
    if ret:
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        H, W, _ = frame.shape
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                cx, cy = int(index_finger.x * W), int(index_finger.y * H)
                Trail.append((cx, cy))
                bbox = landmark_bbox(hand_landmarks, H, W)
                position = normalized_landmarks(hand_landmarks, bbox, H, W)
                # print(position)

                # mp_drawing.draw_landmarks(
                # frame,
                # hand_landmarks,
                # mp_hands.HAND_CONNECTIONS,  
                # mp_drawing_styles.get_default_hand_landmarks_style(),
                # mp_drawing_styles.get_default_hand_connections_style())

        for i in Trail:
            cv.circle(frame, i, 10, (0, 255, 0), cv.FILLED)
        cv.imshow('frame', frame)

        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
video.release()
cv.destroyAllWindows()

(537.1232986450195, 324.9470901489258, 646.2547302246094, 485.26100635528564)
(529.500732421875, 326.102135181427, 646.7144775390625, 489.926118850708)
(524.9024200439453, 328.07806491851807, 646.9822692871094, 494.3150281906128)
(522.0098876953125, 331.46129608154297, 645.5226135253906, 495.3754663467407)
(517.8774261474609, 333.90257835388184, 640.0418853759766, 496.17927074432373)
(514.2151260375977, 335.22274017333984, 636.9302749633789, 495.84916591644287)
(509.4377899169922, 338.7034320831299, 631.5129852294922, 497.7965784072876)
(508.6565399169922, 338.4440088272095, 632.584342956543, 498.570556640625)
(506.8797302246094, 340.5451440811157, 631.2287902832031, 500.2856254577637)
(505.25318145751953, 342.7681374549866, 628.2645797729492, 501.57076835632324)
(503.4132385253906, 344.08458709716797, 624.4915390014648, 500.48616886138916)
(500.7786178588867, 345.84327936172485, 623.0846405029297, 499.90612506866455)
(500.6034851074219, 347.95241832733154, 621.7642211914062, 500.10726